In [66]:
# CONSTANTS
PROJECT_ID = "zennaihackason"
DATASET = "law_db"
TABLE = "personal_info"

In [67]:
from langchain_google_vertexai import VertexAIEmbeddings

# Embedding model
embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko-multilingual@latest", project=PROJECT_ID
)

In [68]:
from google.cloud import bigquery

# BigQuery Client
client = bigquery.Client(project="zennaihackason")

In [69]:
from langchain.vectorstores.utils import DistanceStrategy
from langchain_community.vectorstores import BigQueryVectorSearch

# Vector Store
store = BigQueryVectorSearch(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    embedding=embedding,
    distance_strategy=DistanceStrategy.COSINE,
)

In [70]:
import os
import re
import requests
import numpy as np
from xml.etree import ElementTree
from google.cloud import bigquery


# ----------------------------
# LawLoader クラス（上記のクラス定義）
# ----------------------------
class LawLoader(object):
    """
    Prepare law data with e-Gov (https://www.e-gov.go.jp/) site.

    Args:
        category (int): category number, like 1 (all), 2 (法令), 3 (政令), 4 (省令)
    """

    def __init__(self, category=1):
        self.law_dict = self._get_law_dict(category=category)
        self.content_dict = {}

    @staticmethod
    def _get_xml(url):
        """
        Get XML data from e-Gov API.

        Args:
            url (str): key of the API

        Returns:
            xml.ElementTree: element tree of the XML data
        """
        r = requests.get(url)
        return ElementTree.fromstring(r.content.decode(encoding="utf-8"))

    def _get_law_dict(self, category):
        """
        Return dictionary of law names and numbers.

        Args:
            category (int): category number, like 1 (all), 2 (法令), 3 (政令), 4 (省令)

        Returns:
            dict(str, str): dictionary of law names (keys) and numbers (values)
        """
        url = f"https://elaws.e-gov.go.jp/api/1/lawlists/{category}"
        root = self._get_xml(url)
        names = [e.text for e in root.iter() if e.tag == "LawName"]
        numbers = [e.text for e in root.iter() if e.tag == "LawNo"]
        return {name: num for (name, num) in zip(names, numbers)}

    def get_law_number(self, keyword, category=1):
        """
        Return the law number.
        This will be retrieved from e-Gov (https://www.e-gov.go.jp/)

        Args:
            keyword (str): keyword of the law name
            category (int): category number, like 1 (all), 2 (法令), 3 (政令), 4 (省令)

        Returns:
            dict(str, str): dictionary of law name (key) and law number (value)
        """
        return {k: v for (k, v) in self.law_dict.items() if keyword in k}

    def get_raw(self, number):
        """
        Args:
            number (str): Number of the law, like '平成九年厚生省令第二十八号'

        Returns:
            raw (list[str]): raw contents of J-GCP
        """
        if number in self.content_dict:
            return self.content_dict[number]
        url = f"https://elaws.e-gov.go.jp/api/1/lawdata/{number}"
        root = self._get_xml(url)
        contents = [e.text.strip() for e in root.iter() if e.text]
        raw = [t for t in contents if t]
        self.content_dict[number] = raw
        return raw

    @staticmethod
    def pre_process(raw):
        """
        Perform pre-processing on raw contents.

        Args:
            raw (list[str]): raw contents

        Returns:
            str: pre-processed string

        Notes:
            - Strings enclosed with （ and ） will be removed.
            - 「 and 」 will be removed.
        """
        contents = [s for s in raw if s.endswith("。")]
        string = "".join(contents)
        string = string.translate(str.maketrans({"「": "", "」": ""}))
        return re.sub("（[^（|^）]*）", "", string)

    def gcp(self):
        """
        Perform pre-processing on raw contents of J-GCP.

        Returns:
            str: pre-processed string of J-GCP

        Notes:
            - Article 56 will be removed.
            - Strings enclosed with （ and ） will be removed.
            - 「 and 」 will be removed.
        """
        number_dict = self.get_law_number("医薬品の臨床試験")
        number = number_dict["医薬品の臨床試験の実施の基準に関する省令"]
        raw = self.get_raw(number)
        raw_without56 = raw[: raw.index("第五十六条")]
        return self.pre_process(raw_without56)

In [71]:
loader = LawLoader()

In [72]:
law_numbers = loader.get_law_number("個人情報")
print(len(law_numbers))
print(law_numbers)

25
{'個人情報の保護に関する法律': '平成十五年法律第五十七号', '情報公開・個人情報保護審査会設置法': '平成十五年法律第六十号', '個人情報の保護に関する法律施行令': '平成十五年政令第五百七号', '情報公開・個人情報保護審査会設置法施行令': '平成十五年政令第五百五十号', '個人情報保護委員会事務局組織令': '平成二十七年政令第四百三十四号', '個人情報の保護に関する法律及び行政手続における特定の個人を識別するための番号の利用等に関する法律の一部を改正する法律の施行に伴う関係政令の整備及び経過措置に関する政令': '平成二十八年政令第三百二十四号', '情報公開・個人情報保護審査会事務局組織規則': '平成十七年内閣府令第二十七号', '個人情報の保護に関する法律施行令第二十六条第三項第一号及び第二十九条第三項第一号に掲げる行政機関等が保有する保有個人情報に係る開示請求及び行政機関等匿名加工情報の利用の手続に関する省令': '平成二十五年経済産業省令第十四号', '行政手続における特定の個人を識別するための番号の利用等に関する法律に規定する個人番号、個人番号カード、特定個人情報の提供等に関する命令': '平成二十六年総務省令第八十五号', '個人情報保護委員会事務局組織規則': '平成二十七年内閣府令第七十五号', '行政手続における特定の個人を識別するための番号の利用等に関する法律第十九条第八号に基づく利用特定個人情報の提供に関する命令': '令和六年デジタル庁・総務省令第九号', '会計検査院情報公開・個人情報保護審査会規則': '平成十三年会計検査院規則第三号', '国家公安委員会個人情報管理規則': '平成十七年国家公安委員会規則第五号', '会計検査院の保有する個人情報の保護に関する権限又は事務の委任に関する規則': '平成十七年会計検査院規則第五号', '特定個人情報保護評価に関する規則': '平成二十六年特定個人情報保護委員会規則第一号', '個人情報保護委員会の所管する法令に係る情報通信技術を活用した行政の推進等に関する法律施行規則': '平成二十六年特定個人情報保護委員会規則第二号', '行政手続における特定の個人を識別するための番号の利用等に関する法律第十九条第十七号に基づき同条第十五号に準ずるものとして定める特定個

In [73]:
from tqdm import tqdm 

pre_processed_texts = []
metadatas = []
for law_name, law_number in tqdm(law_numbers.items()):
    raw = loader.get_raw(law_number)
    pre_processed = loader.pre_process(raw)
    
    metadata = {"law_name": law_name, "law_number": law_number}

    pre_processed_texts.append(pre_processed)
    metadatas.append(metadata)

100%|██████████| 25/25 [01:04<00:00,  2.59s/it]


In [74]:
# Insert into BigQuery
# print(metadatas)
store.add_texts(pre_processed_texts, metadatas)

['7ebaf8100d1d4f9faf20e7e5565256f7',
 '4fdd0668e2ed4b8381d11f1908242262',
 'd1b79690edd445a78dea2db1699c9e7c',
 '15865c57ad1840bb90bcee8326438d06',
 '634ae6ee7d704a2bb88c3428a2be25bb',
 '6ba0980ae461499da6594f6e10280fc7',
 '41da32d939d34ac6a45a82f094a24c07',
 '249ce244a09b4408a89e86f74134d336',
 '4dd15a1285d543c58680492ec15cfb48',
 '682b3d64ab9240aea15b510e3676bf7b',
 '07d50110918249faa7b5236f9e7ec6ce',
 '74d529c8fe364b358f0ba81f01ffa35c',
 'f7b8aeca8d0e4f159fea378c838ee310',
 '100e48586abb4e4a8ceafe3e583afc85',
 '7743ce6f58ef44acbdc0b2322b186cbf',
 '5ec94d10a47f4727ab781ec5fd94d91c',
 '71e35eb8f0924c9dbe62d8520f3cc90c',
 'd54bbd4276d141779389a12e3d8910a3',
 'b5c06547bb734595a639aa8f729f5c59',
 '1f95e8a38981419e876b5a90063c7d46',
 '2b0cedfd037342d182310c55f20453e3',
 'df5f5e50f8c440e680651092cdc04b60',
 '2f1322f1e71a4daaa1499b48d7beaf7d',
 '3b29adfee5a946bdbfc293cf79195c3d',
 '34d0437fe03145f585025be3c56ee289']

In [82]:
query = "個人情報保護の適用範囲について"
docs = store.similarity_search(query, k=5)

In [84]:
from pprint import pprint

for doc in docs:
    pprint(len(doc.page_content))
    print("\n")

976


1505


1049


321


1247


